In [65]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

In [48]:
# Importar o dataset
df = pd.read_csv('../data/dataset_problema2.csv')

In [49]:
# Separar features e target
X = df.drop(['id', 'Severidade'], axis=1)
y = df['Severidade']

In [50]:
# Separar dados de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [51]:

# K-fold
kf = KFold(n_splits=10, shuffle=True, random_state=0)

In [52]:
# Pipeline para pre-processamento e modelo
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('rfe', RFE(estimator=RandomForestRegressor())),
    ('regressor', RandomForestRegressor())
])

In [53]:
# Parametros para grid search
param_grid = {
    'rfe__n_features_to_select': range(1, 20),  # Teste de 1 a 20 features
    'regressor__n_estimators': [50, 100, 150]  # Teste de diferentes numeros de arvores no RandomForestRegressor
}

In [54]:
# Grid search para encontrar a melhor combinação de features e hiperparametros
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=kf, scoring='r2', verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 57 candidates, totalling 570 fits


[CV] END regressor__n_estimators=50, rfe__n_features_to_select=1; total time=   4.0s
[CV] END regressor__n_estimators=50, rfe__n_features_to_select=1; total time=   4.0s
[CV] END regressor__n_estimators=50, rfe__n_features_to_select=1; total time=   4.1s
[CV] END regressor__n_estimators=50, rfe__n_features_to_select=1; total time=   4.2s
[CV] END regressor__n_estimators=50, rfe__n_features_to_select=1; total time=   3.9s
[CV] END regressor__n_estimators=50, rfe__n_features_to_select=1; total time=   3.9s
[CV] END regressor__n_estimators=50, rfe__n_features_to_select=1; total time=   4.0s
[CV] END regressor__n_estimators=50, rfe__n_features_to_select=1; total time=   4.0s
[CV] END regressor__n_estimators=50, rfe__n_features_to_select=1; total time=   3.9s
[CV] END regressor__n_estimators=50, rfe__n_features_to_select=1; total time=   4.0s
[CV] END regressor__n_estimators=50, rfe__n_features_to_select=2; total time=   3.9s
[CV] END regressor__n_estimators=50, rfe__n_features_to_select=2;

GridSearchCV(cv=KFold(n_splits=10, random_state=0, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('rfe',
                                        RFE(estimator=RandomForestRegressor())),
                                       ('regressor', RandomForestRegressor())]),
             n_jobs=-1,
             param_grid={'regressor__n_estimators': [50, 100, 150],
                         'rfe__n_features_to_select': range(1, 20)},
             scoring='r2', verbose=2)

In [55]:
# Melhor modelo encontrado pelo grid search
best_model = grid_search.best_estimator_
best_features = grid_search.best_params_['rfe__n_features_to_select']

In [56]:
# Avaliação do modelo final com os dados de teste
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

In [57]:
# Exibindo os parâmetros do melhor modelo
print("\nParâmetros do Melhor Modelo:")
print(best_model.named_steps['regressor'])


Parâmetros do Melhor Modelo:
RandomForestRegressor(n_estimators=50)


In [58]:
# Imprimir as melhores features selecionadas
selected_features = X.columns[best_model.named_steps['rfe'].support_]
print(f'Melhores features selecionadas: {selected_features}')

Melhores features selecionadas: Index(['NDVI_d28', 'SAVI_d28', 'GNDVI_d28', 'SR_d28', 'NDVI_d01', 'GNDVI_d01',
       'NDVI_d04', 'GNDVI_d04', 'MCARI1_d04', 'SR_d04', 'NDVI_d08', 'SAVI_d08',
       'GNDVI_d08', 'MCARI1_d08', 'SR_d08'],
      dtype='object')


In [59]:
# Exibindo os parâmetros do melhor modelo
print("\nParâmetros do Melhor Modelo:")
print(best_model.named_steps['regressor'])


Parâmetros do Melhor Modelo:
RandomForestRegressor(n_estimators=50)


In [67]:
# R2 medio do k-fold
cross_val_scores = cross_val_score(best_model, X_train, y_train, cv=kf, scoring='r2')
print("Pontuação da Validação Cruzada para o Melhor Modelo:")
print(cross_val_scores)
print(f'Média da Pontuação da Validação Cruzada (R2): {np.mean(cross_val_scores):.4f}')

Pontuação da Validação Cruzada para o Melhor Modelo:
[0.98926244 0.68752919 0.88330237 0.44408049 0.92515255 0.32185175
 0.95449818 0.95285412 0.87100179 0.78477882]
Média da Pontuação da Validação Cruzada (R2): 0.7814


In [68]:
# Avaliação do modelo final com os dados de teste
r2 = best_model.score(X_test, y_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print(f'Melhor modelo com {best_features} features:')
print(f'R2: {r2}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')

Melhor modelo com 15 features:
R2: 0.894317103251725
RMSE: 7.311838628679644
MAE: 6.5076370370370356



1. **R2 (Coeficiente de Determinação):**
   - **Significado:** R2 mede o quanto as variações nas previsões do modelo explicam as variações nos dados reais.
   - **Interpretação:** Se o R2 é 1, significa que o modelo prevê os dados perfeitamente. Se é 0, o modelo não explica nada. Valores negativos indicam que o modelo é pior do que uma simples média.

2. **RMSE (Erro Quadrático Médio):**
   - **Significado:** RMSE mede a média dos erros ao quadrado entre as previsões do modelo e os valores reais.
   - **Interpretação:** Quanto menor o RMSE, melhor. Ele representa a média das diferenças entre as previsões do modelo e os dados reais. É uma medida de quão bem o modelo está performando.

3. **MAE (Erro Absoluto Médio):**
   - **Significado:** MAE mede a média das diferenças absolutas entre as previsões do modelo e os valores reais.
   - **Interpretação:** Assim como o RMSE, quanto menor o MAE, melhor. Ele fornece uma média das distâncias entre as previsões e os dados reais, sem levar em conta a direção das diferenças.

Em resumo, o R2 dá uma ideia da proporção de variação explicada, o RMSE mede o tamanho médio dos erros quadráticos, e o MAE mede o tamanho médio dos erros absolutos.